In [ ]:
import os
import sys
from nectarpy import NectarClient

In [ ]:
API_SECRET = "0xfec3f6491a1889f080ef1fdc048027779645fcb3e2051592915cde62876d46fe"

In [ ]:
nectar_client = NectarClient(API_SECRET,'localhost')

In [ ]:
#### Aggregate type: Linear Regression, Mean, Count, Variance, Minimum, Maximum, Sum

In [ ]:
def pre_compute_min_func():
    import pandas as pd
    df = pd.read_csv("/app/data/worker-data.csv")

    return {
        "min_heart_rate": int(df["heart_rate"].min()) if not df.empty else None,
        "min_age": int(df["age"].min()) if not df.empty else None,
    }

In [ ]:
def main_min_func(list_of_partial_result):
    if not list_of_partial_result:
        return {"min_heart_rate": None, "min_age": None}

    min_heart_rate = min(
        (d["min_heart_rate"] for d in list_of_partial_result if d["min_heart_rate"] is not None),
        default=None,
    )
    min_age = min(
        (d["min_age"] for d in list_of_partial_result if d["min_age"] is not None),
        default=None,
    )

    return {"min_heart_rate": min_heart_rate, "min_age": min_age}

In [ ]:
def pre_compute_max_func():
    import pandas as pd
    df = pd.read_csv("/app/data/worker-data.csv")

    return {
        "max_heart_rate": int(df["heart_rate"].max()) if not df.empty else None,
        "max_age": int(df["age"].max()) if not df.empty else None,
    }

In [ ]:

def main_max_func(list_of_partial_result):
    if not list_of_partial_result:
        return {"max_heart_rate": None, "max_age": None}

    max_heart_rate = max(
        (d["max_heart_rate"] for d in list_of_partial_result if d["max_heart_rate"] is not None),
        default=None,
    )
    max_age = max(
        (d["max_age"] for d in list_of_partial_result if d["max_age"] is not None),
        default=None,
    )

    return {"max_heart_rate": max_heart_rate, "max_age": max_age}

In [ ]:
def pre_compute_mean_func():
    import pandas as pd
    import logging as logger
    df = pd.read_csv("/app/data/worker-data.csv")
  
    filtered = df[(df["gender"] == "MALE") & (df["age"] > 50)]
    return {"sum": filtered["heart_rate"].sum(), "count": len(filtered)}

In [ ]:
def main_mean_func(list_of_partial_result):
    total_heart = sum(d.get("sum", 0) for d in list_of_partial_result)
    total_count = sum(d.get("count", 0) for d in list_of_partial_result)
    return {
        "avg_heart_rate": total_heart / total_count if total_count else 0,
        "total_count": total_count
    }

In [ ]:
def pre_compute_variance_func():
    import pandas as pd
    df = pd.read_csv("/app/data/worker-data.csv")
    count = len(df)
    if count == 0:
        return {"count": 0, "sum": 0.0, "sum_sq": 0.0}
    sum_age = df['age'].sum()
    sum_sq_age = (df['age'] ** 2).sum()

    return {"count": count, "sum": sum_age, "sum_sq": sum_sq_age}


In [ ]:
def main_variance_func(list_of_partial_result):
    total_count = sum(d["count"] for d in list_of_partial_result)
    if total_count == 0:
        return {"count": 0, "mean": 0.0, "variance": 0.0}

    total_sum = sum(d["sum"] for d in list_of_partial_result)
    total_sum_sq = sum(d["sum_sq"] for d in list_of_partial_result)

    mean = total_sum / total_count
    # Công thức variance = (Σx² / n) − mean²
    variance = (total_sum_sq / total_count) - (mean ** 2)

    return {"count": total_count, "mean": mean, "variance": variance}

In [ ]:
def pre_compute_linear_regression_func():
    import pandas as pd
    try:
        from sklearn.linear_model import LinearRegression
    except ImportError:
        raise RuntimeError("scikit-learn is not installed in the worker environment")
    
    df = pd.read_csv("/app/data/worker-data.csv")
    df_filtered = df[df['age'] > 10]
    return {
        "X": df_filtered[["heart_rate", "age"]].values.tolist(),
        "Y": df_filtered["height"].values.tolist(),
        "count": len(df_filtered),
    }

In [ ]:
def main_linear_regression_func(list_of_partial_result):
    from sklearn.linear_model import LinearRegression
    import numpy as np

    # Combine X và Y từ tất cả workers
    X_all, Y_all = [], []
    total_count = 0

    for part in list_of_partial_result:
        X_all.extend(part["X"])
        Y_all.extend(part["Y"])
        total_count += part["count"]

    if total_count == 0:
        return {"coef": [], "intercept": 0.0, "count": 0}

    # Train linear regression trên dữ liệu gộp
    X_train = np.array(X_all)
    Y_train = np.array(Y_all)

    model = LinearRegression()
    model.fit(X_train, Y_train)

    return {
        "coef": model.coef_.tolist(),
        "intercept": float(model.intercept_),
        "count": total_count,
    }

In [ ]:
result = nectar_client.byoc_query(
    pre_compute_func = pre_compute_min_func,
    main_func = main_min_func, 
    is_separate_data = True,
    bucket_ids=[59460897943160968540229985522129525620969349443467291153379717412661629982620,8091486222544721738525070139839786624633525609819295572211758262310044448555],
    policy_indexes=[0,0]
)

In [ ]:
result